<a href="https://colab.research.google.com/github/nespar7/Spring_24/blob/main/NLP/Assignments/Assignment_1/NLP_A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installations

In [4]:
!pip install spacy --force-reinstall
!python -m spacy download en_core_web_sm
!pip install contextualSpellCheck
!pip install scikit-learn
!pip install nltk

  Using cached spacy-3.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.9/156.9 kB 9.6 MB/s eta 0:00:00
  Using cached thinc-8.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (922 kB)
  Using cached srsly-2.4.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (493 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl (17 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 15.7 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.3.1
    Uninstalling en-core-web-sm-2.3.1:
      Successfully uninstalled en-core-web-sm-2.3.1
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## Imports

In [5]:
import spacy
import contextualSpellCheck
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from nltk import download as nltk_dw
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk_dw('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Models

In [6]:
nlp = spacy.load("en_core_web_sm")
contextualSpellCheck.add_to_pipe(nlp)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Function Definitions

In [7]:
def spell_check(text):
    doc = nlp(text)

    return doc._.performed_spellCheck, doc._.outcome_spellCheck

In [8]:
def correct_spellings(csv_file, out_csv_file, column_name, should_log=False):
    # read a csv file, in each line, see if spellCheck was performed(if text was correctly spelled) and print original and corrected texts
    lines = []

    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        lines = list(reader)

        # Index of the column to be spell checked
        column_index = lines[0].index(column_name)

        # For each line, perform spell check
        for i in range(1, len(lines)):
            line = lines[i][column_index]
            performed, outcome = spell_check(line)


            # If spell check was performed, update the line and print the original and corrected texts
            if performed:
                # Update the line
                lines[i][column_index] = outcome

                if should_log:
                    print(line)
                    print(outcome)
                    print()

            if i%100 == 0:
                print(i)

    with open(out_csv_file, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(lines)

    print("Spell check completed")

In [9]:
def pre_process(csv_file, out_csv_file, column_name):
    # read a csv file, in each line, remove characters apart from alphanumerics and whitespaces in the given column
    lines = []

    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        lines = list(reader)

        # Index of the column to be pre processed
        column_index = lines[0].index(column_name)

        for i in range(1, len(lines)):
            lines[i][column_index] = ''.join([c for c in lines[i][column_index] if c.isalnum() or c.isspace()])

    # write the pre-processed data to the resulting csv file
    with open(out_csv_file, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(lines)

    print("Pre procesing completed")

In [10]:
def get_vocabulary(model, csv_file, column_name):
    # Read the spell checked docs csv file and store the documents in a list
    docs = []

    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        lines = [line for line in reader]

        doc_col_index = lines[0].index(column_name)

        for i in range(1, len(lines)):
            docs.append(lines[i][doc_col_index])

        num_docs = len(docs)
        lower_lim = 5
        upper_lim = int(num_docs * 0.85)

        # tokenize the documents using spacy and store the tokens in a vocabulary,
        # remove the tokens that occur less than lower_lim times and more than upper_lim times

        doc_tokenized = []

        # use the model to tokenize the documents
        for doc in docs:
            # process doc using the model passed
            doc = model(doc)

            # Select alphabetical tokens and remove stop words
            tokens = [token.text.lower() for token in doc if token.is_alpha and not token.is_stop]

            doc_tokenized.append(tokens)

        # Now that we have the tokenized documents, we can create a vocabulary
        vocabulary = {}

        for doc in doc_tokenized:
            for token in doc:
                if token in vocabulary:
                    vocabulary[token] += 1
                else:
                    vocabulary[token] = 1

        # Remove the tokens that occur less than lower_lim times and more than upper_lim times
        vocabulary = [k for k, v in vocabulary.items() if v >= lower_lim and v <= upper_lim]

        return vocabulary

In [40]:
def get_processed_text(model, csv_file, col_name):
    # Read the csv file and return the processed text using the passed model
    lines = []
    docs = []

    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        lines = list(reader)

        col_index = lines[0].index(col_name)

        for i in range(1, len(lines)):
            doc = model(lines[i][col_index])
            docs.append(' '.join([token.text.lower() for token in doc]))

    return docs

In [49]:
def get_stemmed_vocabulary(csv_file, col_name):
    # Read the spell checked docs csv file and store the documents in a list
    docs = []

    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        lines = [line for line in reader]

        doc_col_index = lines[0].index(col_name)

        for i in range(1, len(lines)):
            docs.append(lines[i][doc_col_index])

    # tokenize the documents using nltk and store the tokens in a vocabulary,
    # remove the tokens that occur less than lower_lim times and more than upper_lim times

    lower_lim = 5
    upper_lim = int(len(docs) * 0.85)

    doc_tokenized = []

    # use nltk to tokenize the documents
    for doc in docs:
        doc = word_tokenize(doc)

        tokens = [token.lower() for token in doc if token.isalpha()]

        # stem the tokens
        stemmed_tokens = [ps.stem(token) for token in tokens]

        doc_tokenized.append(stemmed_tokens)

    vocabulary = {}

    for doc in doc_tokenized:
        for token in doc:
            if token in vocabulary:
                vocabulary[token] += 1
            else:
                vocabulary[token] = 1

    # Remove the tokens that occur less than lower_lim times and more than upper_lim times
    stemmed_vocabulary = [k for k, v in vocabulary.items() if v >= lower_lim and v <= upper_lim]

    return stemmed_vocabulary

In [44]:
def get_stemmed_texts(csv_file, column_name, porter_stemmer):
    # Read the spell checked docs csv file and store the queries in a list
    texts = []

    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        lines = [line for line in reader]

        col_index = lines[0].index(column_name)

        for i in range(1, len(lines)):
            texts.append(lines[i][col_index])

    stemmed_texts = []

    # use nltk to tokenize the documents
    for text in texts:
        text = word_tokenize(text)

        tokens = [token.lower() for token in text]

        # stem the tokens
        stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]

        stemmed_texts.append(' '.join(stemmed_tokens))

    return stemmed_texts

In [14]:
def get_texts(csv_file, id_col, text_col):
    # Read the spell checked docs csv file and store the documents in a list
    ids = []
    texts = []

    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        lines = [line for line in reader]

        id_col_index = lines[0].index(id_col)
        text_col_index = lines[0].index(text_col)

        for i in range(1, len(lines)):
            ids.append(lines[i][id_col_index])
            texts.append(lines[i][text_col_index])

    return ids, texts

In [15]:
def get_relevant_docs(qdrels):
    # Get relevant docs list for each query from qdrels.csv file
    relevant_docs = {}

    with open(qdrels, 'r') as f:
        reader = csv.reader(f)
        lines = [line for line in reader]

        for i in range(1, len(lines)):
            line = lines[i]

            query_id = line[1]
            doc_id = line[2]

            if query_id in relevant_docs:
                relevant_docs[query_id].append(doc_id)
            else:
                relevant_docs[query_id] = [doc_id]

    return relevant_docs

In [54]:
def precision_k_score(query_id_idx_map, relevant_docs, top_docs, k):
    # For each query in relevant_docs, calculate true positives in top_docs and return average precision_k score
    precision_k = 0

    for query_id, relevant_docs_list in relevant_docs.items():
        true_positives = 0

        query_idx = query_id_idx_map[query_id]

        for doc in top_docs[query_idx]:
            if doc in relevant_docs_list:
                true_positives += 1

        precision_k += true_positives / k

    return precision_k / len(relevant_docs)

## Main

The CSV file locations need to be changed as applicable if being run locally.

In [19]:
docs_file = "/content/Query_Doc/docs.csv"
queries_file = "/content/Query_Doc/queries.csv"
pre_processed_docs_file = "/content/Query_Doc/pre_processed_docs.csv"
pre_processed_queries_file = "/content/Query_Doc/pre_processed_queries.csv"
spell_checked_docs_file = "/content/Query_Doc/spell_checked_docs.csv"
spell_checked_queries_file = "/content/Query_Doc/spell_checked_queries.csv"
qdrels_file = "/content/Query_Doc/qdrel.csv"

Pre processing

In [ ]:
pre_process(docs_file, pre_processed_docs_file, "doc_text")
pre_process(queries_file, pre_processed_queries_file, "query_text")

Spell correction on the pre processed docs and queries

In [ ]:
# Print logs(original and corrected for query doc only)
correct_spellings(pre_processed_queries_file, spell_checked_queries_file, "query_text", True)
correct_spellings(pre_processed_docs_file, spell_checked_docs_file, "doc_text")

Tokenize the documents and get the tokens forming the vocabulary

In [37]:
# Remove contextualSpellCheck, lemmatizer, ner and tagger from the nlp pipe since we are checking performance after only spell check
nlp.remove_pipe('contextual spellchecker')
nlp.remove_pipe('tagger')
nlp.remove_pipe('lemmatizer')
nlp.remove_pipe('ner')

('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x7aa82ab77f80>)

In [20]:
vocabulary = get_vocabulary(nlp, spell_checked_docs_file, "doc_text")

In [21]:
print(vocabulary)
print(len(vocabulary))

['step', 'invest', 'share', 'market', 'india', 'story', 'happen', 'indian', 'government', 'increase', 'speed', 'internet', 'connection', 'increased', 'mentally', 'solve', 'find', 'water', 'sugar', 'salt', 'carbon', 'survive', 'moon', 'sun', 'm', 'buy', 'active', 'far', 'phone', 'video', 'games', 'good', 'great', 'use', 'instead', 'company', 'code', 'files', 'free', 'method', 'atoms', 'things', 'tell', 'components', 'read', 'youtube', 'comments', 'physics', 'easy', 'learn', 'sexual', 'experience', 'like', 'laws', 'change', 'status', 'student', 'visa', 'green', 'card', 'compare', 'immigration', 'canada', 'japan', 'trump', 'presidency', 'mean', 'current', 'international', 'masters', 'students', 'eu', 'affect', 'planning', 'study', 'means', 'girls', 'want', 'friends', 'guy', 'guys', 'feel', 'girl', 'quora', 'users', 'questions', 'answered', 'google', 'people', 'ask', 'easily', 'best', 'digital', 'marketing', 'bangladesh', 'institute', 'look', 'white', 'bombs', 's', 'causing', 'jealous', 'a

In [22]:
doc_ids, docs = get_texts(spell_checked_docs_file, 'doc_id', 'doc_text')
query_ids, queries = get_texts(spell_checked_queries_file, 'query_id', 'query_text')

query_id_idx_map = {query_ids[i]: i for i in range(len(query_ids))}
doc_id_idx_map = {doc_ids[i]: i for i in range(len(doc_ids))}

In [23]:
relevant_docs = get_relevant_docs(qdrels_file)

In [24]:
print(query_id_idx_map)

{'4584': 0, '6588': 1, '10113': 2, '7957': 3, '5498': 4, '7614': 5, '7301': 6, '4974': 7, '2615': 8, '8950': 9, '9658': 10, '2429': 11, '1803': 12, '318': 13, '6055': 14, '7841': 15, '9733': 16, '4874': 17, '2824': 18, '744': 19, '9889': 20, '7822': 21, '10024': 22, '4105': 23, '2791': 24, '7988': 25, '378': 26, '8808': 27, '11167': 28, '2757': 29, '7319': 30, '9294': 31, '5799': 32, '5647': 33, '8810': 34, '6483': 35, '3796': 36, '5040': 37, '6253': 38, '3076': 39, '9965': 40, '8456': 41, '9615': 42, '399': 43, '7372': 44, '1578': 45, '8831': 46, '3459': 47, '540': 48, '1956': 49, '3017': 50, '6209': 51, '10155': 52, '8055': 53, '3322': 54, '2658': 55, '6936': 56, '548': 57, '6276': 58, '609': 59, '1350': 60, '10188': 61, '2785': 62, '3521': 63, '9312': 64, '420': 65, '3959': 66, '2822': 67, '2145': 68, '2678': 69, '784': 70, '5162': 71, '7396': 72, '1248': 73, '7463': 74, '1453': 75, '8458': 76, '7222': 77, '5516': 78, '1992': 79, '9376': 80, '9214': 81, '11591': 82, '4356': 83, '585

Vectorize documents and queries, and find cosine similarities for each query with the documents

In [25]:
vectorizer = TfidfVectorizer(vocabulary=vocabulary)

# Create tf idf vectors for the documents and queries
doc_vectors = vectorizer.fit_transform(docs)
query_vectors = vectorizer.transform(queries)

In [26]:
similarities = cosine_similarity(query_vectors, doc_vectors)

In [27]:
# find top 5 and top 10 similar document indices for each query
top_5_indices = np.argsort(similarities, axis=1)[:, -5:]
top_10_indices = np.argsort(similarities, axis=1)[:, -10:]

# for each query, store the top 5 and top 10 similar document ids
top_5_ids = [[doc_ids[i] for i in indices] for indices in top_5_indices]
top_10_ids = [[doc_ids[i] for i in indices] for indices in top_10_indices]
top_ids = [[i[-1]] for i in top_5_ids]

In [28]:
prec_1 = precision_k_score(query_id_idx_map, relevant_docs, top_ids, 1)
prec_5 = precision_k_score(query_id_idx_map, relevant_docs, top_5_ids, 5)
prec_10 = precision_k_score(query_id_idx_map, relevant_docs, top_10_ids, 10)

print(f"Average precision at 1: {round(prec_1 * 100, 2)} %")
print(f"Average precision at 5: {round(prec_5 * 100, 2)} %")
print(f"Average precision at 10: {round(prec_10 * 100, 2)} %")

Average precision at 1: 51.0 %
Average precision at 5: 16.2 %
Average precision at 10: 8.7 %


## Task 2

### Stemming

Using nltk's porter stemmer here since spacy does not provide a stemmer

In [29]:
ps = PorterStemmer()

In [50]:
stemmed_vocabulary = get_stemmed_vocabulary(spell_checked_docs_file, "doc_text")
stemmed_docs = get_stemmed_texts(spell_checked_docs_file, "doc_text", ps)
stemmed_queries = get_stemmed_texts(spell_checked_queries_file, "query_text", ps)

print(stemmed_vocabulary)
print(len(stemmed_vocabulary))

['what', 'is', 'the', 'step', 'by', 'to', 'invest', 'in', 'share', 'market', 'india', 'stori', 'of', 'would', 'happen', 'if', 'indian', 'govern', 'back', 'how', 'can', 'i', 'increas', 'speed', 'my', 'internet', 'connect', 'while', 'use', 'a', 'be', 'pass', 'through', 'whi', 'am', 'mental', 'veri', 'solv', 'it', 'find', 'when', 'that', 'which', 'one', 'water', 'sugar', 'salt', 'and', 'carbon', 'fish', 'surviv', 'moon', 'sun', 'doe', 'say', 'about', 'me', 'im', 'not', 'thi', 'should', 'buy', 'keep', 'her', 'activ', 'far', 'from', 'phone', 'video', 'game', 'good', 'do', 'great', 'you', 'instead', 'compani', 'have', 'code', 'file', 'for', 'free', 'method', 'separ', 'atom', 'are', 'some', 'thing', 'tell', 'reliabl', 'laptop', 'compon', 'read', 'youtub', 'comment', 'see', 'all', 'make', 'physic', 'easi', 'learn', 'wa', 'your', 'first', 'sexual', 'experi', 'like', 'law', 'chang', 'statu', 'student', 'visa', 'green', 'card', 'us', 'they', 'compar', 'immigr', 'canada', 'japan', 'trump', 'presid

Vectorize the documents and queries, and find cosine similarities for each query with documents

In [51]:
vectorizer_stemmed = TfidfVectorizer(vocabulary=stemmed_vocabulary)

# Create tf idf vectors for the documents and queries
stemmed_doc_vectors = vectorizer_stemmed.fit_transform(stemmed_docs)
stemmed_query_vectors = vectorizer_stemmed.transform(stemmed_queries)

In [52]:
stemmed_similarities = cosine_similarity(stemmed_query_vectors, stemmed_doc_vectors)

# find top 5 and top 10 similar document indices for each query
stemmed_top_5_indices = np.argsort(stemmed_similarities, axis=1)[:, -5:]
stemmed_top_10_indices = np.argsort(stemmed_similarities, axis=1)[:, -10:]

# for each query, store the top 5 and top 10 similar document ids
stemmed_top_5_ids = [[doc_ids[i] for i in indices] for indices in stemmed_top_5_indices]
stemmed_top_10_ids = [[doc_ids[i] for i in indices] for indices in stemmed_top_10_indices]
stemmed_top_ids = [[ids[-1]] for ids in stemmed_top_5_ids]

Precision Scores

In [55]:
prec_1_stemmed = precision_k_score(query_id_idx_map, relevant_docs, stemmed_top_ids, 1)
prec_5_stemmed = precision_k_score(query_id_idx_map, relevant_docs, stemmed_top_5_ids, 5)
prec_10_stemmed = precision_k_score(query_id_idx_map, relevant_docs, stemmed_top_10_ids, 10)

print(f"Average precision at 1 for stemmed: {round(prec_1_stemmed * 100, 2)} %")
print(f"Average precision at 5 for stemmed: {round(prec_5_stemmed * 100, 2)} %")
print(f"Average precision at 10 for stemmed: {round(prec_10_stemmed * 100, 2)} %")

Average precision at 1 for stemmed: 61.0 %
Average precision at 5 for stemmed: 19.0 %
Average precision at 10 for stemmed: 10.4 %


## Lemmatization

Add lemmatizer back to the nlp pipe

In [38]:
nlp.add_pipe('lemmatizer')
nlp.initialize()

In [56]:
lemmatized_vocabulary = get_vocabulary(nlp, spell_checked_docs_file, "doc_text")
lemmatized_docs = get_processed_text(nlp, spell_checked_docs_file, "doc_text")
lemmatized_queries = get_processed_text(nlp, spell_checked_queries_file, "query_text")

print(len(lemmatized_vocabulary))

/usr/local/lib/python3.10/dist-packages/spacy/pipeline/attributeruler.py:149: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)
/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


2086


Vectorize the documents and queries, and find cosine similarities for each query with documents

In [ ]:
vectorizer_lemmatized = TfidfVectorizer(vocabulary=lemmatized_vocabulary)

# Create tf idf vectors for the documents and queries
lemmatized_doc_vectors = vectorizer_lemmatized.fit_transform(lemmatized_docs)
lemmatized_query_vectors = vectorizer_lemmatized.transform(lemmatized_queries)